In [30]:
import os, time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def download_money_market_excel(
    from_date="2025-05-01",
    to_date="2025-08-18",
    download_dir=".",
    headless=False,
    wait_timeout=30
):
    os.makedirs(download_dir, exist_ok=True)

    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    prefs = {"download.default_directory": os.path.abspath(download_dir)}
    opts.add_experimental_option("prefs", prefs)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)

    try:
        url = "https://www.cbsl.lk/eResearch/Modules/RD/SearchPages/Indicators_MoneyMarketSummary.aspx"
        driver.get(url)

        # wait for input fields
        from_selector = (By.ID, "ContentPlaceHolder1_txtFrom")
        to_selector = (By.ID, "ContentPlaceHolder1_txtTo")
        wait = WebDriverWait(driver, wait_timeout)
        wait.until(EC.presence_of_element_located(from_selector))
        wait.until(EC.presence_of_element_located(to_selector))

        driver.find_element(*from_selector).clear()
        driver.find_element(*from_selector).send_keys(from_date)
        time.sleep(3)  # Wait 3 seconds after entering from date

        driver.find_element(*to_selector).clear()
        driver.find_element(*to_selector).send_keys(to_date)
        time.sleep(3)  # Wait 3 seconds after entering to date

        # click "Show" button (try known IDs)
        for btn_id in ["ContentPlaceHolder1_btnShow", "ContentPlaceHolder1_btnSearch"]:
            els = driver.find_elements(By.ID, btn_id)
            if els:
                els[0].click()
                break
        else:
            driver.find_element(*to_selector).send_keys("\n")

        # wait for 1 minute for page to render results
        print("Waiting 1 minute for page to render results...")
        time.sleep(60)

        # click the Export to Excel button
        export_id = "ContentPlaceHolder1_btnExport"
        export_button = driver.find_element(By.ID, export_id)
        export_button.click()

        # wait for download to complete
        before = set(os.listdir(download_dir))
        timeout = time.time() + wait_timeout
        downloaded_file = None
        while time.time() < timeout:
            after = set(os.listdir(download_dir))
            new = after - before
            for fname in new:
                if fname.lower().endswith((".xls", ".xlsx", ".csv")):
                    downloaded_file = os.path.join(download_dir, fname)
                    break
            if downloaded_file:
                break
            time.sleep(0.5)

        if not downloaded_file:
            raise RuntimeError(f"Download timed out. No Excel file found in {download_dir}")

        print("Downloaded file:", downloaded_file)
        return downloaded_file
    finally:
        driver.quit()

# Example usage:
if __name__ == "__main__":
    excel_path = download_money_market_excel(headless=False, wait_timeout=60)
    print("Excel saved at:", excel_path)

Waiting 1 minute for page to render results...


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0xcaba83+63395]
	GetHandleVerifier [0x0xcabac4+63460]
	(No symbol) [0x0xaf2113]
	(No symbol) [0x0xae1b20]
	(No symbol) [0x0xaff922]
	(No symbol) [0x0xb65c9c]
	(No symbol) [0x0xb80099]
	(No symbol) [0x0xb5f1a6]
	(No symbol) [0x0xb2e7b2]
	(No symbol) [0x0xb2f654]
	GetHandleVerifier [0x0xf28883+2672035]
	GetHandleVerifier [0x0xf23cba+2652634]
	GetHandleVerifier [0x0xcd2bca+223466]
	GetHandleVerifier [0x0xcc2cb8+158168]
	GetHandleVerifier [0x0xcc978d+185517]
	GetHandleVerifier [0x0xcb3b78+96408]
	GetHandleVerifier [0x0xcb3d02+96802]
	GetHandleVerifier [0x0xc9e90a+9770]
	BaseThreadInitThunk [0x0x769f5d49+25]
	RtlInitial

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def show_money_market(from_date="2025-05-01", to_date="2025-08-18", headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    wait = WebDriverWait(driver, 30)

    try:
        url = "https://www.cbsl.lk/eResearch/Modules/RD/SearchPages/Indicators_MoneyMarketSummary.aspx"
        driver.get(url)

        # Fill 'From' date
        from_input = wait.until(EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_txtFrom")))
        from_input.clear()
        from_input.send_keys(from_date)
        print("Entered From date:", from_date)
        time.sleep(3)

        # Fill 'To' date
        to_input = driver.find_element(By.ID, "ContentPlaceHolder1_txtTo")
        to_input.clear()
        to_input.send_keys(to_date)
        print("Entered To date:", to_date)
        time.sleep(3)

        # Click 'Show' button (try known IDs) or press Enter
        for btn_id in ["ContentPlaceHolder1_btnShow", "ContentPlaceHolder1_btnSearch"]:
            els = driver.find_elements(By.ID, btn_id)
            if els:
                els[0].click()
                print("Clicked Show button")
                break
        else:
            to_input.send_keys("\n")
            print("Pressed Enter instead of Show button")

        print("Page should now display results.")

    finally:
        # Keep browser open for inspection (optional)
        input("Press Enter to quit browser...")
        driver.quit()


# Example usage:
if __name__ == "__main__":
    show_money_market("2025-08-01", "2025-08-18", headless=False)



Entered From date: 2025-08-01
Entered To date: 2025-08-18
Clicked Show button
Page should now display results.
